### Tensorflow Tutorials

##### What is Tensorflow?

- graph nodes are operations
- graph edges are tensors which flow between nodes

##### Basic Concpets

- *Variables*: stateful nodes which output their current value (parameters)(by definition, operation)
- *Placeholders* : nodes whose value is fed in at execution time (inputs, labels)

##### Flows

1. Define Graphs
2. Run through a session

In [1]:
import tensorflow as tf
import numpy as np

/home/hojoon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/hojoon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### H = ReLU(Wx+b)

In [2]:
# Build Graph

b = tf.Variable(tf.zeros((10,)))
W = tf.Variable(tf.random_uniform((50, 10), -1, 1))

x = tf.placeholder(tf.float32, (100, 50))

h = tf.nn.relu(tf.matmul(x,W) + b)

In [3]:
tf.get_default_graph().get_operations()

[<tf.Operation 'zeros' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'random_uniform/shape' type=Const>,
 <tf.Operation 'random_uniform/min' type=Const>,
 <tf.Operation 'random_uniform/max' type=Const>,
 <tf.Operation 'random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'random_uniform/sub' type=Sub>,
 <tf.Operation 'random_uniform/mul' type=Mul>,
 <tf.Operation 'random_uniform' type=Add>,
 <tf.Operation 'Variable_1' type=VariableV2>,
 <tf.Operation 'Variable_1/Assign' type=Assign>,
 <tf.Operation 'Variable_1/read' type=Identity>,
 <tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'MatMul' type=MatMul>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'Relu' type=Relu>]

In [4]:
# Run the Graph 
# sess.run(fetches, feeds)
# fetches: output, feeds: input
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables()) #we can assign values only on run time
    sess.run(h, {x: np.random.randn(100, 50)}) #insert feed dictionary with numpy array

Instructions for updating:
Use `tf.global_variables_initializer` instead.


#### Training the Model

In [5]:
# Use placeholders for labels
# Build loss node using labels and prediction

prediction = tf.nn.softmax(h)
label = tf.placeholder(tf.float32, [None,10])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(label * tf.log(prediction), 
                                              reduction_indices = [1]))

In [6]:
# tf.train.GradientdescentOptimizer is an Optimizer object
# tf.train.GradientDescentOptimizer(lr).minimize(loss) optimizes computation graph

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [9]:
# Create session -> Build training schedule -> Run train_step
# sess.run(train_step, feeds)

lab = np.ones((100,10))
lab[np.arange(100), np.random.choice(np.arange(10), 100)] = 1

sess = tf.Session()
sess.run(tf.initialize_all_variables())

for i in range(100):
    sess.run(train_step, feed_dict = {x: np.random.randn(100, 50), label: lab})

#### Variable Sharing

In [10]:
# naive way: use variables saving in dictionary
# Use Name
#   - tf.variable_scope() : provides simple name-spacing to avoid crashes
#   - tf.get_variable()   : creates.accesses variables from within variable scope

with tf.variable_scope("foo"):
    v = tf.get_variable("v", shape = [1]) # v.name == "foo/v:0"
with tf.variable_scope("foo", reuse= True):
    v1 = tf.get_variable("v")  # Shared variable found!